In [ ]:
# Financial Analysis Application

In this notebook, you will create agents to review financial documents and generate a Market Price Analysis (MPA) using the CrewAI framework.

## Installation

Install the required packages:
```sh
!pip install -r requirements.txt

import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Agent: Document Reviewer
doc_reviewer = Agent(
    role="Document Reviewer",
    goal="Review financial statements and extract key financial figures.",
    backstory="You're tasked with reviewing financial statements provided by a potential seller. Your goal is to extract and provide key financial figures needed for further analysis.",
    allow_delegation=False,
    verbose=True
)

# Agent: Financial Analyst
financial_analyst = Agent(
    role="Financial Analyst",
    goal="Analyze the extracted financial figures and provide detailed insights.",
    backstory="You're tasked with analyzing the financial figures extracted by the Document Reviewer. Your goal is to provide detailed insights into the financial health of the business.",
    allow_delegation=False,
    verbose=True
)

# Agent: Addback Analyst
addback_analyst = Agent(
    role="Addback Analyst",
    goal="Analyze the granular numbers associated with addbacks and provide details.",
    backstory="You're tasked with analyzing the granular numbers associated with addbacks provided by the seller. Your goal is to provide detailed insights into these addbacks.",
    allow_delegation=False,
    verbose=True
)

# Agent: MPA Preparer
mpa_preparer = Agent(
    role="MPA Preparer",
    goal="Prepare a Market Price Analysis (MPA) and list questions for the seller.",
    backstory="You're tasked with preparing the Market Price Analysis (MPA) using the insights provided by the Financial Analyst and Addback Analyst. Your goal is to compile this into a comprehensive report and generate a list of questions where the data is questionable.",
    allow_delegation=False,
    verbose=True
)

# Agent: Editor
editor = Agent(
    role="Editor",
    goal="Review the MPA and questions to ensure accuracy and clarity.",
    backstory="You're tasked with reviewing the Market Price Analysis (MPA) and the list of questions generated by the MPA Preparer. Your goal is to ensure accuracy and clarity in the final documents.",
    allow_delegation=False,
    verbose=True
)

# Task: Review Documents
review_docs = Task(
    description=(
        "1. Review the financial statements provided by the seller.\n"
        "2. Extract key financial figures such as revenue, profit, expenses, etc.\n"
        "3. Provide these figures in a structured format."
    ),
    expected_output="Structured financial figures extracted from the documents.",
    agent=doc_reviewer,
)

# Task: Analyze Financial Figures
analyze_financials = Task(
    description=(
        "1. Analyze the extracted financial figures.\n"
        "2. Provide detailed insights into the financial health of the business."
    ),
    expected_output="Detailed financial analysis report.",
    agent=financial_analyst,
)

# Task: Analyze Addbacks
analyze_addbacks = Task(
    description=(
        "1. Analyze the granular numbers associated with addbacks.\n"
        "2. Provide detailed insights into these addbacks."
    ),
    expected_output="Detailed addback analysis report.",
    agent=addback_analyst,
)

# Task: Prepare MPA
prepare_mpa = Task(
    description=(
        "1. Compile the insights from the financial and addback analyses.\n"
        "2. Prepare a comprehensive Market Price Analysis (MPA).\n"
        "3. Generate a list of questions where the data is questionable."
    ),
    expected_output="Comprehensive MPA report with a list of questions.",
    agent=mpa_preparer,
)

# Task: Review MPA
review_mpa = Task(
    description=(
        "1. Review the MPA and the list of questions.\n"
        "2. Ensure accuracy and clarity in the final documents."
    ),
    expected_output="Final reviewed MPA report and list of questions.",
    agent=editor,
)

crew = Crew(
    agents=[doc_reviewer, financial_analyst, addback_analyst, mpa_preparer, editor],
    tasks=[review_docs, analyze_financials, analyze_addbacks, prepare_mpa, review_mpa],
    verbose=2
)

result = crew.kickoff(inputs={"topic": "User Supplied Document"})

# Display the results
from IPython.display import Markdown
Markdown(result)

